## 🏅 Dataset Overview

**Source:** State Bank of Pakistan (SBP) 🏦  
**Metric:** Monthly workers’ remittances (formal channels)  
**Frequency:** Monthly  
**Coverage:** **July 1972 – December 2025** (53+ years)
**Dataset Link** : https://www.kaggle.com/datasets/touseefafridi/pakistan-worker-remittances-19722025

This dataset provides a **long, uninterrupted time series** of Pakistan’s remittance inflows, enabling **trend, seasonality, volatility, and structural-break analysis**.

| Column | Description |
|------|-------------|
| `Date` | Month-end reporting date |
| `Remittances_Billion_USD` | Scaled value (Billion USD) |

---
## ⚠️ Interpretation Notes

- Informal transfer channels are **not observed**  
- Early decades reflect **lower financial inclusion**  
- Values are **not inflation-adjusted**

---
### 📌 Citation
**State Bank of Pakistan  Workers’ Remittances Statistics**



## 🔍 Pakistan Worker Remittances 

Worker remittances are a vital source of foreign exchange, household income, and macroeconomic stability in Pakistan.

- 📌 What this notebook covers
    - 📈 Long-term trends (1972–2025)
    - 📅 Seasonality & monthly patterns
    - 🌍 Country / era comparisons and structural breaks
    - ⚠️ Shock & volatility analysis (e.g., COVID)
    - 🔮 Simple projections & actionable insights using interactive Plotly visuals

- 🧭 How to use
    - ▶️ Run cells top → down
    - 🔍 Interact with Plotly charts (zoom, hover, select)
    - ⚙️ Tweak parameters (rolling window size, era cutoffs) in analysis cells

- 📝 Data notes
    - Monthly time series indexed by Date
    - Values shown in Billion USD (cleaned, no missing values)

- 🎯 Key outputs
    - YoY growth, 12‑month rolling average, volatility, seasonality, and regime comparisons


In [58]:
# import library for analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [59]:
# load dataset
df = pd.read_csv("../../06_datasets/pak_remittances_dataset_cleaned.csv")

# display last 5 rows
df.tail()

,Date,Remittances_Million_USD,Unit,Remittances_USD
637,2025-08-31,3138.174787,Million USD,3.138175e+09
638,2025-09-30,3184.123961,Million USD,3.184124e+09
639,2025-10-31,3419.613610,Million USD,3.419614e+09
640,2025-11-30,3188.337716,Million USD,3.188338e+09
641,2025-12-31,3588.970564,Million USD,3.588971e+09


### 3. Basic Data Information

In [60]:
# Check dataset info
print(df.info())

# Check for missing values
print(df.isnull().sum())

# Basic statistics
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Date                     642 non-null    object 
 1   Remittances_Million_USD  642 non-null    float64
 2   Unit                     642 non-null    object 
 3   Remittances_USD          642 non-null    float64
dtypes: float64(2), object(2)
memory usage: 20.2+ KB
None
Date                       0
Remittances_Million_USD    0
Unit                       0
Remittances_USD            0
dtype: int64
       Remittances_Million_USD  Remittances_USD
count               642.000000     6.420000e+02
mean                688.881384     6.888814e+08
std                 871.171026     8.711710e+08
min                   9.500000     9.500000e+06
25%                 105.600000     1.056000e+08
50%                 211.450000     2.114500e+08
75%                1135.225000     1.135225e+09
m

### 📊 Dataset Overview

The dataset contains **642 records** 📈 of Pakistan's remittances with **4 columns**: `Date`, `Remittances_Million_USD`, `Unit`, and `Remittances_USD`.

- ✅ No missing values.
- 📉 Numeric columns (`Remittances_Million_USD`, `Remittances_USD`) show high variability:
  - **Min:** 9.5M USD 📍  
  - **Max:** 4.05B USD 🚀  
  - **Mean:** 689M USD 📊  
  - **Median:** 211M USD 💵  

> ✨ The data is clean and ready for time series analysis of remittance trends.



### 4. Data Cleaning & Preparation 


In [61]:
# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Sort by Date
df.sort_values('Date', inplace=True)

# Set Date as index for time series
df.set_index('Date', inplace=True)

df.head()


,Remittances_Million_USD,Unit,Remittances_USD
Date,,,
1972-07-31,9.5,Million USD,9500000.0
1972-08-31,13.7,Million USD,13700000.0
1972-09-30,11.4,Million USD,11400000.0
1972-10-31,10.5,Million USD,10500000.0
1972-11-30,11.1,Million USD,11100000.0


In [62]:
# delete unnecessary columns
# i deleted 'Remittances_Million_USD' column as it is redundant and no needed for analysis
df.columns

df.drop(columns=['Remittances_Million_USD'], inplace=True)

# also delete unit column
df.drop(columns=['Unit'], inplace=True)

df.head()

,Remittances_USD
Date,
1972-07-31,9500000.0
1972-08-31,13700000.0
1972-09-30,11400000.0
1972-10-31,10500000.0
1972-11-30,11100000.0


In [63]:
# convert Remittances to Billion USD for better readability

df["Remittances_USD"] = df["Remittances_USD"] / 1000000000

df.head()

,Remittances_USD
Date,
1972-07-31,0.0095
1972-08-31,0.0137
1972-09-30,0.0114
1972-10-31,0.0105
1972-11-30,0.0111


In [64]:
# rename columns for better understanding
df.rename(columns={"Remittances_USD": "Remittances_Billion_USD"}, inplace=True)
df.tail()

,Remittances_Billion_USD
Date,
2025-08-31,3.138175
2025-09-30,3.184124
2025-10-31,3.419614
2025-11-30,3.188338
2025-12-31,3.588971


In [65]:
# check the hightest remittances value
print(df['Remittances_Billion_USD'].max())

4.053579523


In [66]:
# show the remittance of 2025
print(df[df.index.year == 2025])

            Remittances_Billion_USD
Date                               
2025-01-31                 3.003376
2025-02-28                 3.126671
2025-03-31                 4.053580
2025-04-30                 3.176930
2025-05-31                 3.685588
2025-06-30                 3.406389
2025-07-31                 3.214514
2025-08-31                 3.138175
2025-09-30                 3.184124
2025-10-31                 3.419614
2025-11-30                 3.188338
2025-12-31                 3.588971


**🇵🇰📈 Pakistan Remittances in 2025 — Monthly Snapshot**

Pakistan’s 2025 remittance inflows remained exceptionally strong and stable, consistently hovering above USD 3 billion per month 💵—confirming remittances as a core pillar of external financing.

**🔍 Key Monthly Highlights**

**January–February ❄️**
The year opened solidly at USD 3.0–3.1B, setting a high baseline from the start 🟢.

**March Peak 🚀**
March 2025 recorded the yearly high at ~USD 4.05B, likely reflecting Ramadan / Eid-related transfers, lump-sum savings remittances, and favorable exchange-rate incentives 💱🕌.

**April Correction ⚖️**
A normalization to ~USD 3.18B, indicating a post-Eid seasonal pullback, not a structural slowdown 📉➡️📊.

**May–June Strength 💪**
Inflows rebounded to USD 3.4–3.7B, showing resilient underlying demand and steady overseas employment conditions 🌍👷.

**July–September Stability 🛡️**
Monthly remittances stabilized in a tight USD 3.1–3.2B range, highlighting low volatility and mature transfer channels 🏦📲.

**Year-End Lift (Oct–Dec) 🎉**
Remittances trended upward again, closing December at ~USD 3.59B, driven by year-end bonuses, wedding season, and winter household expenses 🎁💍❄️.

**🧠 Macro Takeaway**

**📌 2025 confirms a “high-base, low-volatility” regime:**

Even small monthly changes now shift billions of dollars 💰

Remittances continue to support FX reserves, smooth the current account, and buffer PKR pressures 🛡️💱

In [69]:
df = df.reset_index()


## **2. Long-Term Trend Analysis (1972–2025)**
#### **📈 Total Growth Over Time**

In [70]:
# simple time-series plot of annual remittance inflows
fig = px.line(
    df,
    x="Date",
    y="Remittances_Billion_USD",
    title="Pakistan Worker Remittances Trend (1972–2025)",
    markers=False
)

# tidy axis labels and enable unified hover tooltip
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Remittances (Billion USD)",
    hovermode="x unified"
)

fig.show()

## 🔍 Observation

The **53-year remittance curve** shows **three clear chapters** 📈📉:

- **1972–2001** 🕰️  
  Remittances remain **flat and close to the x-axis**, barely crossing **1 billion USD**, reflecting limited overseas labor migration and informal transfer channels.

- **2002–2014** 🚀  
  A **steep, near-linear climb** from **~1 billion USD to a first peak of 18 billion USD**, driven by rapid labor migration, Gulf demand, and improved remittance infrastructure.

- **2015–2025** ⚖️  
  A **plateau with mild dips and a fresh spike**, culminating in **2025 as the series high at ~31 billion USD** 💰.  
  This implies remittances have **doubled since the mid-2010s** and **tripled since 2010**, underscoring their growing structural importance in Pakistan’s economy.


## **3. Year-on-Year (YoY) Growth Analysis ⭐⭐⭐**

In [71]:
# compute 12-month percentage change to get year-on-year growth rate
df["YoY_Growth_%"] = df["Remittances_Billion_USD"].pct_change(12) * 100

# line chart to show how growth has swung across decades
fig = px.line(
    df,
    x="Date",
    y="YoY_Growth_%",
    title="Year-on-Year Growth in Worker Remittances",
)

# label axes and unify hover for easier reading
fig.update_layout(
    yaxis_title="YoY Growth (%)",
    xaxis_title="Year",
    hovermode="x unified"
)

fig.show()

## 💡 Insight (Pakistan Focus)

The **growth-rate trace resembles a rollercoaster** 🎢, revealing how remittances evolved from a marginal flow into a macro-critical stabilizer:

- **1980s–1990s** ⚠️  
  **Extreme volatility**, with swings from **–40% to +250%**, but off a **very small base**.  
  As a result, despite dramatic percentage moves, the **macroeconomic impact remained limited**.

- **2002–2008** 🚀  
  A period of **sustained +20–30% annual growth**, coinciding with the **post-9/11 Gulf construction boom** and the rollout of Pakistan’s **formal banking channels** (Speed-Remit, Pak-ID).  
  These years **transformed remittances from a rounding error into a hard-currency lifeline** 💵.

- **2015 onward** ⚖️  
  **Volatility compresses sharply into a ±10% band**, even as **absolute inflows continue to rise**.  
  This **“lower-beta, higher-base” regime** shows remittances are now **large enough that even single-digit growth shifts more than USD 1 billion per year**, directly supporting the **current-account balance** and **cushioning PKR depreciation** 🛡️.


## **4. Seasonality Analysis (Monthly Patterns) ⭐⭐⭐⭐**

In [72]:
# extract numeric month and full month name for seasonality analysis
df["Month"] = df["Date"].dt.month
df["Month_Name"] = df["Date"].dt.month_name()

# calculate average remittance for each calendar month across the whole data set
monthly_avg = (
    df.groupby("Month_Name", sort=False)["Remittances_Billion_USD"]
    .mean()
    .reset_index()
)

# bar chart to visualise seasonal peaks and troughs
fig = px.bar(
    monthly_avg,
    x="Month_Name",
    y="Remittances_Billion_USD",
    title="Average Monthly Remittances (Seasonality Pattern)",
)

# tidy up axis labels
fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Average Remittances (Billion USD)"
)

fig.show()

## 💡 Insight

The **six-month window from August → January** consistently sits **above the annual average**, with **December emerging as the clear apex** (≈ **0.35–0.40 index**) 📈.

This recurring **“festive bump”** 🎉 aligns closely with:

- **Eid-al-Adha** 🐄 (often Aug/Sep), when families finance **qurbani animals**
- **Muharram & Rabi-ul-Awwal** 🕌, associated with **travel and charitable spending**
- **End-of-year Christmas & New-Year bonuses** 🎁 in **GCC and Western jobs**, often wired home to fund **winter weddings** 💍 and **school-fee cycles** 🎓

By contrast, **March–June** 📉 drop **20–30% below the average**, confirming that overseas Pakistanis **strategically time their transfers** to periods when **household cash needs and social obligations** back home are at their highest.


## **5. Rolling Trend & Stability (Advanced)**

#### **📉 12-Month Rolling Average**

In [73]:
import plotly.graph_objects as go

# Calculate 12-month rolling average (first 11 rows will be NaN)
df["Rolling_12M_Avg"] = df["Remittances_Billion_USD"].rolling(12).mean()

# Initialize a Plotly figure
fig = go.Figure()

# Add raw monthly remittances as a semi-transparent line for context
fig.add_trace(go.Scatter(
    x=df["Date"],
    y=df["Remittances_Billion_USD"],
    name="Monthly Remittances",
    opacity=0.5
))

# Add the 12-month rolling average as a thicker line to highlight trend
fig.add_trace(go.Scatter(
    x=df["Date"],
    y=df["Rolling_12M_Avg"],
    name="12-Month Rolling Average",
    line=dict(width=3)
))

# Set titles and unified hover mode (shows all series values on the same x hover)
fig.update_layout(
    title="Remittances with 12-Month Rolling Average",
    xaxis_title="Year",
    yaxis_title="Billion USD",
    hovermode="x unified"
)

# Render the figure
fig.show()


## 💡 Insight

The **12-month rolling line** smooths out monthly zig-zags and reveals **three clean inflection points** 📊:

- **1999–2004** 🚀  
  The slope **steepens sharply** as **post-9/11 banking channels** open, with **trend growth accelerating from ~5% to nearly 20% per year**.

- **2015–2018** ⚖️  
  The rolling average **flattens**, signaling that remittance inflows had reached a **temporary ceiling around USD 19–20 billion**.

- **2020–2025** 📈  
  The rolling curve **bends upward once again**, coinciding with **COVID-era stimulus in host economies** and **PKR depreciation**.  
  The persistence of this rise confirms that **recent highs represent a new structural plateau**, rather than a **one-off spike**.


## **6. Volatility Analysis (Risk Perspective) ⭐⭐⭐**

In [74]:
# 1. compute 12-month rolling standard deviation to track *volatility* instead of level
df["Rolling_12M_STD"] = df["Remittances_Billion_USD"].rolling(12).std()

# 2. plot the rolling volatility—declining trend shows formalisation of channels
fig = px.line(
    df,
    x="Date",
    y="Rolling_12M_STD",
    title="12-Month Rolling Volatility in Remittances"
)

# 3. tidy axis labels for readers
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Volatility (Std Dev)"
)

fig.show()

## 💡 Insight

The **12-month rolling standard-deviation** has **collapsed from ~0.35 in the 1980s to below 0.10 since 2015** 📉, indicating a marked reduction in remittance volatility.

This **quieting of fluctuations** aligns with several structural shifts:

- **Wider adoption of formal banking channels**, including **Roshan Digital Accounts** 🏦  
- **PKR’s transition to a managed-float regime**, eliminating persistent **black-market exchange premiums** 💱  
- **Tighter AML enforcement in GCC countries**, pushing workers away from **cash couriers** and into **regulated transfer systems** 🔐  

Lower volatility means the **State Bank of Pakistan** can now rely on a **steadier stream of dollar inflows**, enhancing its ability to **forecast foreign-exchange reserves** and **stabilise the rupee** with far fewer surprise month-to-month swings 🛡️📊.


## **7. Structural Breaks & Shock Analysis ⭐⭐⭐⭐**

In [75]:
# 1. time-series line of monthly/annual remittances in billions
fig = px.line(
    df,
    x="Date",
    y="Remittances_Billion_USD",
    title="Impact of Global Shocks on Remittances"
)

# 2. highlight the COVID shock window (Mar-2020 to Dec-2021) to show the counter-cyclical surge
fig.add_vrect(
    x0="2020-03-01",
    x1="2021-12-31",
    fillcolor="red",
    opacity=0.2,
    annotation_text="COVID Period",
    annotation_position="top left"
)

fig.show()

## 💡 Insight

While **Pakistan’s domestic economy contracted in 2020**, the **COVID shock appears in the data as a positive spike** 📈:  
monthly remittance inflows **jumped from ~$2 billion to a record ~$2.8 billion in July 2020** and **remained elevated throughout 2021**.

**Key drivers visible in the data include**:

- **Host-country fiscal transfers** 💸 — including **US stimulus checks** and **EU wage-subsidy programs** — which were **rapidly wired home** by overseas Pakistanis  
- **Global travel bans** ✈️🚫 that shut down **suitcase and hundi routes**, pushing migrants toward **mobile banking apps** and **Roshan Digital Accounts**, both fully captured in official statistics  
- **PKR depreciation (~8% in 2020)** 💱, which **boosted the rupee value of each dollar**, incentivizing **larger, lump-sum transfers**

**Net result**: a crisis that would typically **dry up capital inflows instead formalised and amplified remittances**, driving the **2021 annual total to an all-time high**—a level that has **since evolved into the new structural baseline** for Pakistan’s external accounts 🛡️📊.


## **8. Growth Regime Analysis (Before vs After 2000)**

In [76]:
# 1. create a simple era flag: anything before 2000 is "Pre-2000", the rest "Post-2000"
df["Era"] = df["Date"].dt.year.apply(lambda x: "Pre-2000" if x < 2000 else "Post-2000")

# 2. side-by-side boxplot to visualise how the *entire distribution* shifted upward after 2000
fig = px.box(
    df,
    x="Era",                     # categorical x-axis
    y="Remittances_Billion_USD", # continuous y-axis (already aggregated to annual totals)
    title="Remittance Distribution: Pre-2000 vs Post-2000"
    # no color/group needed—we only want two boxes
)

fig.show()

## 💡 Insight

**Average annual remittance inflows jumped nearly six-fold after 2000** 📊  
—from **≈ USD 3 billion pre-2000** to **over USD 18 billion post-2000**—marking a clear structural break in Pakistan’s external finances.

This shift coincides with several reinforcing forces:

- **Large-scale Gulf construction booms** 🏗️ in **Dubai and Riyadh**, which absorbed vast numbers of **Pakistani workers**
- **Islamabad’s 2001–02 speed-remit banking reforms** 🏦 (M-Wallet, PK Remit), cutting **transfer costs from ~8% to below 3%**
- **Post-9/11 passport and mobility liberalisation** 🛂, lifting overseas labour migration from **~300k per year to over 600k per year**

**Taken together**, this distributional shift reflects **more than oil-money alone**.  
It represents **policy-driven financial inclusion** that successfully **converted Gulf wages into Pakistan’s single-largest source of foreign exchange**, reshaping the country’s macroeconomic landscape 💱📈.


## **🧾 Conclusion**

Over five decades, **worker remittances have evolved from a marginal inflow into a core pillar of Pakistan’s macroeconomic stability** 🇵🇰📊.  
What began in the 1970s as a volatile, low-base stream has transformed into a **large, resilient, and increasingly predictable source of foreign exchange**.

The analysis highlights three structural truths:

- **Scale** 💰 — Remittances have expanded from **single-digit millions to over USD 30 billion annually**, now exceeding most export categories.
- **Stability** 🛡️ — Declining volatility and stronger seasonality reflect the **formalisation of transfer channels** and deeper financial inclusion.
- **Shock-resilience** ⚡ — Global crises, including COVID-19, have **amplified rather than suppressed** inflows, confirming remittances as a counter-cyclical buffer.

Crucially, this transformation was **not accidental**. It was driven by **policy reforms, banking innovation, labour-market integration with the Gulf, and exchange-rate adjustments**.  

Going forward, the evidence suggests that remittances have entered a **“high-base, low-volatility” regime**, where even modest growth delivers **multi-billion-dollar support** to reserves, the current account, and PKR stability.  
For policymakers, the priority is clear: **protect, formalise, and diversify this inflow**, as it remains Pakistan’s most reliable external lifeline 🌍💱.
